# 005 - Ensemble / Blending

## Objectif
Combiner les prédictions de **RUN 003 (XGBoost + Feature Engineering)** et **RUN 004 (CatBoost)**.

### Pourquoi blender ?
Si les modèles font des erreurs différentes (non corrélées), leur moyenne pondérée tend à réduire la variance globale et l'erreur.
- **XGBoost (RUN 003)** : Très performant (LB: 14125), utilise des features ingéniées.
- **CatBoost (RUN 004)** : Performant (CV: 15264), utilise nativement les variables catégorielles.

Nous allons tester plusieurs poids, en favorisant le modèle le plus fort (XGBoost).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
%matplotlib inline

## 1. Chargement des Prédictions

In [ ]:
SUB_XGB_PATH = "../submissions/submission_003_fe_tuned_xgb.csv"
SUB_CAT_PATH = "../submissions/submission_004_catboost.csv"

df_xgb = pd.read_csv(SUB_XGB_PATH)
df_cat = pd.read_csv(SUB_CAT_PATH)

# Vérification de l'alignement
assert df_xgb["Id"].equals(df_cat["Id"]), "Erreur d'alignement des IDs !"

print("Shape:", df_xgb.shape)

## 2. Analyse de la Corrélation
Si la corrélation est très proche de 1, le blend apportera peu. Si elle est plus basse (< 0.95), le gain potentiel est plus grand.

In [ ]:
plt.figure(figsize=(8, 8))
plt.scatter(df_xgb["SalePrice"], df_cat["SalePrice"], alpha=0.5, s=10)
plt.plot([0, 800000], [0, 800000], 'r--')
plt.xlabel("XGBoost Predictions")
plt.ylabel("CatBoost Predictions")
plt.title("Comparaison des prédictions")
plt.show()

corr = np.corrcoef(df_xgb["SalePrice"], df_cat["SalePrice"])[0, 1]
print(f"Corrélation entre les prédictions : {corr:.4f}")

## 3. Création des Blends
Nous générons plusieurs fichiers avec des poids différents pour XGBoost (0.9, 0.8, 0.7, 0.6).

In [ ]:
weights_xgb = [0.9, 0.8, 0.7, 0.6]
output_dir = "../submissions"

for w in weights_xgb:
    w_cat = 1.0 - w
    blend_pred = w * df_xgb["SalePrice"] + w_cat * df_cat["SalePrice"]
    
    filename = f"submission_005_blend_w{int(w*10):02d}.csv"
    # filepath = f"{output_dir}/{filename}" # Utiliser chemin relatif dans notebook
    
    # On sauvegarde juste pour info, le script .py fait aussi le travail
    # submission = pd.DataFrame({"Id": df_xgb["Id"], "SalePrice": blend_pred})
    # submission.to_csv(filepath, index=False)
    
    print(f"Blend {w*100:.0f}% XGB / {w_cat*100:.0f}% Cat generated.")